# Import libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# !pip install https://github.com/scikit-learn-contrib/scikit-learn-extra/archive/master.zip
# !pip uninstall -y enum34
# !pip install scikit-learn-extra
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import random
from sklearn.cluster import Birch, AgglomerativeClustering, KMeans, MiniBatchKMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
# from sklearn_extra.cluster import KMedoids

from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # Read data and clean

In [ ]:
df = pd.read_csv('../input/fifa-20-complete-player-dataset/players_20.csv')
col_list = np.r_[4:5, 6:8, 10:14, 16:19, 22:23, 31:43, 45:78]
df_numeric = df.iloc[:, col_list]
df_numeric = df_numeric.fillna(0)
df = df_numeric
# df.drop(['Unnamed: 0'], inplace=True, axis=1)
df.drop(['goalkeeping_diving'], inplace=True, axis=1)
df.drop(['goalkeeping_handling'], inplace=True, axis=1)
df.drop(['goalkeeping_kicking'], inplace=True, axis=1)
df.drop(['goalkeeping_positioning'], inplace=True, axis=1)
df.drop(['goalkeeping_reflexes'], inplace=True, axis=1)

In [ ]:
from sklearn.decomposition import PCA
# Standardize the data to have a mean of ~0 and a variance of 1
X_std = StandardScaler().fit_transform(df)
# Create a PCA instance: pca
pca = PCA(n_components=0.95)
principalComponents = pca.fit_transform(X_std)
# Plot the explained variances
features = range(pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color='black')
plt.xlabel('PCA features')
plt.ylabel('variance %')
plt.xticks(features)
# Save components to a DataFrame
PCA_components = pd.DataFrame(principalComponents)

In [ ]:
dtaf = pd.read_csv('/kaggle/input/fifa-20-complete-player-dataset/players_20.csv')
# dtaf = dtaf.select_dtypes([np.number])
col_list = np.r_[4:5, 6:8, 10:14, 16:19, 22:23, 31:43, 45:78]
dtaf = dtaf.iloc[:, col_list]
dtaf.isna().sum() 


In [ ]:
df = pd.read_csv('/kaggle/input/newdataset/players_20_numeric_normalized-2.csv')
df.dataframeName = 'players_20_numeric_normalized.csv'

#clean
df.drop(['Unnamed: 0'], inplace=True, axis=1)
df.drop(['goalkeeping_diving'], inplace=True, axis=1)
df.drop(['goalkeeping_handling'], inplace=True, axis=1)
df.drop(['goalkeeping_kicking'], inplace=True, axis=1)
df.drop(['goalkeeping_positioning'], inplace=True, axis=1)
df.drop(['goalkeeping_reflexes'], inplace=True, axis=1)
rowNum, colNum = df.shape
print(f"There are {rowNum} rows and {colNum} cols")

In [ ]:
from sklearn.preprocessing import StandardScaler
from collections import Counter
df_o = df[df['gk_speed']==0]
# print(df_gk)

df_gk = df[df['gk_speed']!=0]
NUM_GOALKEEPER = df_gk.shape[0]
# print(df_o)
print(NUM_GOALKEEPER)

from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# # calculate the distance from each point to its closest neighbor
# nn = NearestNeighbors(n_neighbors = 50)

# # fit the nearest neighbor
# nbrs = nn.fit(df_o)

# # returns two arrays - distance to the closest n_neighbors points and index for each point
# distances, indices = nbrs.kneighbors(df_o)

# # sort the distance and plot it
# distances = np.sort(distances, axis=0)
# distances = distances[:,1]
# plt.plot(distances)
db = DBSCAN(eps=1, min_samples=4).fit(StandardScaler().fit_transform(PCA_components.iloc[:,:3]))

# db = DBSCAN(eps=3, min_samples=4).fit(StandardScaler().fit_transform(df_o))

# df_concat = pd.concat([df_gk, df_o]);
# gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=3, dtype=np.int)
# labels_concat = np.concatenate((gk_labels, db.labels_))

# print(db.labels_.value_counts())
print(Counter(db.labels_))
# print(len(set(db.labels_)) - (1 if -1 in db.labels_ else 0))
# print(metrics.calinski_harabasz_score(df_o,db.labels_))
# print(metrics.calinski_harabasz_score(df_concat, labels_concat))
# print(metrics.davies_bouldin_score(df_concat, labels_concat))

In [ ]:
print(metrics.calinski_harabasz_score(df, db.labels_))
print(metrics.davies_bouldin_score(df, db.labels_))

> # Exploration:

# correlation matrix

In [ ]:
print(df.columns)

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()
plotCorrelationMatrix(df, 15)

> # Clustering

# splitting goalkeeper & non-goalkepper

In [ ]:
df_gk = df[df['gk_speed']!=0]
NUM_GOALKEEPER = df_gk.shape[0]
# print(df_o)
print(NUM_GOALKEEPER)

df_o = df[df['gk_speed']==0]
# print(df_gk)
NUM_OTHERS = df_o.shape[0]
print(NUM_OTHERS)

0. K-MEANS

In [ ]:
clust_model = KMeans(n_clusters=4)
clust_model.fit(df)
labels = clust_model.labels_
print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))

In [ ]:
for n in range(2, 10):
    clust_model = KMeans(n_clusters=n)
    clust_model.fit(df)
    labels = clust_model.labels_
    print(n)
    print(metrics.calinski_harabasz_score(df, labels))
    print(metrics.davies_bouldin_score(df, labels))

In [ ]:
from sklearn.cluster import Birch, AgglomerativeClustering, KMeans, MiniBatchKMeans
clust_model = MiniBatchKMeans(n_clusters=4)
clust_model.fit(df)
labels = clust_model.labels_
print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))

In [ ]:
from sklearn.cluster import kmeans_plusplus

In [ ]:
#df => df_o, df_gk
clust_model = KMeans(n_clusters=3)
clust_model.fit(df_o)
labels = clust_model.labels_

df_concat = pd.concat([df_gk, df_o]);
gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=3, dtype=np.int)
labels_concat = np.concatenate((gk_labels, labels))
# print(type(gk_labels))
# print(gk_labels.shape)
# print(type(labels))
# print(labels.shape)

print(metrics.calinski_harabasz_score(df_o, labels))
print(metrics.davies_bouldin_score(df_o, labels))
print(metrics.calinski_harabasz_score(df_concat, labels_concat))
print(metrics.davies_bouldin_score(df_concat, labels_concat))

1. BIRCH

In [ ]:
import scipy.cluster.hierarchy as sch
dendrogrm = sch.dendrogram(sch.linkage(df_o, method = 'ward', metric='euclidean'))
plt.title('Dendrogram')
plt.xlabel('Players')
plt.ylabel('Euclidean distance')
plt.show()

best #cluster seems to be 3

In [ ]:
#df => df_o, df_gk
clust_model = Birch(n_clusters=2)
clust_model.fit(df_o)
labels = clust_model.labels_

df_concat = pd.concat([df_gk, df_o]);
gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=3, dtype=np.int)
labels_concat = np.concatenate((gk_labels, labels))
# print(type(gk_labels))
# print(gk_labels.shape)
# print(type(labels))
# print(labels.shape)

print(metrics.calinski_harabasz_score(df_o, labels))
print(metrics.davies_bouldin_score(df_o, labels))
print(metrics.calinski_harabasz_score(df_concat, labels_concat))
print(metrics.davies_bouldin_score(df_concat, labels_concat))

In [ ]:
clust_model.get_params()

2. AHC

In [ ]:
#df => df_o, df_gk
clust_model = AgglomerativeClustering(n_clusters=3)
clust_model.fit(df_o)
labels = clust_model.labels_

df_concat = pd.concat([df_gk, df_o]);
gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=3, dtype=np.int)
labels_concat = np.concatenate((gk_labels, labels))
# print(type(gk_labels))
# print(gk_labels.shape)
# print(type(labels))
# print(labels.shape)

print(metrics.calinski_harabasz_score(df_o, labels))
print(metrics.davies_bouldin_score(df_o, labels))
print(metrics.calinski_harabasz_score(df_concat, labels_concat))
print(metrics.davies_bouldin_score(df_concat, labels_concat))

In [ ]:
3. CLARA

In [ ]:
class ClaraClustering(object):
    """The clara clustering algorithm.

    Basically an iterative guessing version of k-mediods that makes things a lot faster
    for bigger data sets.
    """

    def __init__(self, max_iter=100000):
        """Class initialization.

        :param max_iter: The default number of max iterations
        """
        self.max_iter = max_iter
        self.dist_cache = dict()

    def clara(self, _df, _k, _fn):
        """The main clara clustering iterative algorithm.

        :param _df: Input data frame.
        :param _k: Number of medoids.
        :param _fn: The distance function to use.
        :return: The minimized cost, the best medoid choices and the final configuration.
        """
        size = len(_df)
        if size > 100000:
            niter = 1000
            runs = 1
        else:
            niter = self.max_iter
            runs = 5

        min_avg_cost = np.inf
        best_choices = []
        best_results = {}

        for j in range(runs):
            sampling_idx = random.sample([i for i in range(size)], (40+_k*2))
            sampling_data = []
            for idx in sampling_idx:
                sampling_data.append(_df.iloc[idx])

            sampled_df = pd.DataFrame(sampling_data)
            pre_cost, pre_choice, pre_medoids = self.k_medoids(sampled_df, _k, _fn, niter)
            tmp_avg_cost, tmp_medoids = self.average_cost(_df, _fn, pre_choice)
            if tmp_avg_cost <= min_avg_cost:
                min_avg_cost = tmp_avg_cost
                best_choices = list(pre_choice)
                best_results = dict(tmp_medoids)

        return min_avg_cost, best_choices, best_results

    def k_medoids(self, _df, _k, _fn, _niter):
        """The original k-mediods algorithm.

        :param _df: Input data frame.
        :param _k: Number of medoids.
        :param _fn: The distance function to use.
        :param _niter: The number of iterations.
        :return: Cluster label.

        Pseudo-code for the k-mediods algorithm.
        1. Sample k of the n data points as the medoids.
        2. Associate each data point to the closest medoid.
        3. While the cost of the data point space configuration is decreasing.
            1. For each medoid m and each non-medoid point o:
                1. Swap m and o, recompute cost.
                2. If global cost increased, swap back.
        """
        print('K-medoids starting')
        # Do some smarter setting of initial cost configuration
        pc1, medoids_sample = self.cheat_at_sampling(_df, _k, _fn, 17)
        prior_cost, medoids = self.compute_cost(_df, _fn, medoids_sample)
        current_cost = prior_cost
        iter_count = 0
        best_choices = []
        best_results = {}

        print('Running with {m} iterations'.format(m=_niter))
        while iter_count < _niter:
            for m in medoids:
                clust_iter = 0
                for item in medoids[m]:
                    if item != m:
                        idx = list(medoids_sample).index(m)
                        swap_temp = medoids_sample[idx]
                        medoids_sample[idx] = item
                        tmp_cost, tmp_medoids = self.compute_cost(_df, _fn, medoids_sample, True)
                        if (tmp_cost < current_cost) & (clust_iter < 1):
                            best_choices = list(medoids_sample)
                            best_results = dict(tmp_medoids)
                            current_cost = tmp_cost
                            clust_iter += 1
                        else:
                            best_choices = best_choices
                            best_results = best_results
                            current_cost = current_cost
                        medoids_sample[idx] = swap_temp

            iter_count += 1
            if best_choices == medoids_sample:
                print('Best configuration found!')
                break

            if current_cost <= prior_cost:
                prior_cost = current_cost
                medoids = best_results
                medoids_sample = best_choices

        return current_cost, best_choices, best_results

    def compute_cost(self, _df, _fn, _cur_choice, cache_on=True):
        """A function to compute the configuration cost.

        :param _df: The input data frame.
        :param _fn: The distance function.
        :param _cur_choice: The current set of medoid choices.
        :param cache_on: Binary flag to turn caching.
        :return: The total configuration cost, the mediods.
        """
        size = len(_df)
        total_cost = 0.0
        medoids = {}
        for idx in _cur_choice:
            medoids[idx] = []

        for i in range(size):
            choice = -1
            min_cost = np.inf
            for m in medoids:
                if cache_on:
                    tmp = self.dist_cache.get((m, i), None)

                if not cache_on or tmp is None:
                    if _fn == 'manhattan':
                        tmp = self.manhattan_distance(_df.iloc[m], _df.iloc[i])
                    elif _fn == 'cosine':
                        tmp = self.cosine_distance(_df.iloc[m], _df.iloc[i])
                    elif _fn == 'euclidean':
                        tmp = self.euclidean_distance(_df.iloc[m], _df.iloc[i])
                    elif _fn == 'fast_euclidean':
                        tmp = self.fast_euclidean(_df.iloc[m], _df.iloc[i])
                    else:
                        print('You need to input a distance function.')

                if cache_on:
                    self.dist_cache[(m, i)] = tmp

                if tmp < min_cost:
                    choice = m
                    min_cost = tmp

            medoids[choice].append(i)
            total_cost += min_cost

        return total_cost, medoids

    def average_cost(self, _df, _fn, _cur_choice):
        """A function to compute the average cost.

        :param _df: The input data frame.
        :param _fn: The distance function.
        :param _cur_choice: The current medoid candidates.
        :return: The average cost, the new medoids.
        """
        _tc, _m = self.compute_cost(_df, _fn, _cur_choice)
        avg_cost = _tc / len(_m)
        return avg_cost, _m

    def cheat_at_sampling(self, _df, _k, _fn, _nsamp):
        """A function to cheat at sampling for speed ups.

        :param _df: The input data frame.
        :param _k: The number of mediods.
        :param _fn: The distance function.
        :param _nsamp: The number of samples.
        :return: The best score, the medoids.
        """
        size = len(_df)
        score_holder = []
        medoid_holder = []
        for _ in range(_nsamp):
            medoids_sample = random.sample([i for i in range(size)], _k)
            prior_cost, medoids = self.compute_cost(_df, _fn, medoids_sample, True)
            score_holder.append(prior_cost)
            medoid_holder.append(medoids)

        idx = score_holder.index(min(score_holder))
        ms = medoid_holder[idx].keys()
        return score_holder[idx], ms

    def euclidean_distance(self, v1, v2):
        """Slow function for euclidean distance.

        :param v1: The first vector.
        :param v2: The second vector.
        :return: The euclidean distance between v1 and v2.
        """
        dist = 0
        for a1, a2 in zip(v1, v2):
            dist += abs(a1 - a2)**2
        return dist

    def fast_euclidean(self, v1, v2):
        """Faster function for euclidean distance.

        :param v1: The first vector.
        :param v2: The second vector.
        :return: The euclidean distance between v1 and v2.
        """
        return np.linalg.norm(v1 - v2)

    def manhattan_distance(self, v1, v2):
        """Function for manhattan distance.

        :param v1: The first vector.
        :param v2: The second vector.
        :return: The manhattan distance between v1 and v2.
        """
        dist = 0
        for a1, a2 in zip(v1, v2):
            dist += abs(a1 - a2)
        return dist

    def cosine_distance(self, v1, v2):
        """Function for cosine distance.

        :param v1: The first vector.
        :param v2: The second vector.
        :return: The cosine distance between v1 and v2.
        """
        xx, yy, xy = 0, 0, 0
        for a1, a2 in zip(v1, v2):
            xx += a1*a1
            yy += a2*a2
            xy += a1*a2
        return float(xy) / np.sqrt(xx*yy)

In [ ]:
def clara_new(_df,_k):
    """The main clara clustering iterative algorithm.

        :param _df: Input data frame.
        :param _k: Number of medoids.
        :return: The minimized cost, the best medoid choices and the final configuration.
        """
    size = len(_df)
    if size > 100000:
        niter = 1000
        runs = 1
    else:
        niter = 100000
        runs = 5
        # runs = 2

    min_avg_cost = np.inf
    best_choices = []
    best_results = {}
    
    for j in range(runs):
        sampling_idx = random.sample([i for i in range(size)], (40+_k*2))
        sampling_data = []
        for idx in sampling_idx:
            sampling_data.append(_df.iloc[idx])
        
        sampled_df = pd.DataFrame(sampling_data)
        cobj = KMedoids(n_clusters=_k,random_state=0,max_iter=niter).fit(sampled_df)
        pre_choice = cobj.medoid_indices_
        print(pre_choice);
        tmp_avg_cost, tmp_medoids = Clara.average_cost(_df, 'euclidean', pre_choice)
#         print("debug 1: "+str(tmp_avg_cost));
        # print("debug 2:" + str(tmp_medoids))
        result = dict(tmp_medoids)
#         print(type(result[pre_choice[0]]))
#         print(len(result[pre_choice[0]]))
        if tmp_avg_cost <= min_avg_cost:
            min_avg_cost = tmp_avg_cost
            best_choices = list(pre_choice)
            best_results = dict(tmp_medoids)
    return min_avg_cost, best_choices, best_results

In [ ]:
num_cluster = 4
result = clara_new(df_o,num_cluster)

best_choices = np.full(shape=NUM_OTHERS, fill_value = 0,dtype=np.int)
for cluster_index in range(num_cluster):
    for index in result[2][result[1][cluster_index]]:
        best_choices[index] = cluster_index

print(sk.metrics.calinski_harabasz_score(df,best_choices))
print(sk.metrics.davies_bouldin_score(df,best_choices))

Checking

In [ ]:
print(df.isna().sum())
print(df.columns)
df.head(5)
df['goalkeeping_handling'].value_counts

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
#     df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
#     print(nCol)
#     print(nGraphShown)
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
plotPerColumnDistribution(df, 57, 5)

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()
plotScatterMatrix(df, 20, 10)

Dendogram

In [ ]:
import scipy.cluster.hierarchy as sch
dendrogrm = sch.dendrogram(sch.linkage(df, method = 'ward'))
plt.title('Dendrogram')
plt.xlabel('Players')
plt.ylabel('Euclidean distance')
plt.show()

# Clustering

# Birch

In [ ]:
Birch-clustering full dataset 2 clusters

In [ ]:
clust_model = Birch(n_clusters=2)
clust_model.fit(df)
labels = clust_model.labels_
print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))
print(metrics.silhouette_score(df, labels))

Birch-clustering full dataset 4 cluster

In [ ]:
clust_model = Birch(n_clusters=4)
clust_model.fit(df)
labels = clust_model.labels_
print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))

In [ ]:
10211.76140489249
1.6618000246511475

In [ ]:
print(labels)
#print(labels.value_counts)
from collections import Counter

Counter(labels)

Birch-clustering separate goal-keeper 3 clusters

Birch-clustering separate goal-keeper 5 clusters

In [ ]:
#df => df_o, df_gk
clust_model = Birch(n_clusters=5)
clust_model.fit(df_o)
labels = clust_model.labels_

df_concat = pd.concat([df_gk, df_o]);
gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=5, dtype=np.int)
labels_concat = np.concatenate((gk_labels, labels))

print(metrics.calinski_harabasz_score(df_concat, labels_concat))
print(metrics.davies_bouldin_score(df_concat, labels_concat))

# AHC

In [ ]:
AHC fulldataset 2 clusters

In [ ]:
clust_model = AgglomerativeClustering(n_clusters=2)
clust_model.fit(df)
labels = clust_model.labels_

print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))
print(metrics.silhouette_score(df, labels))

AHC full dataset 4 clusters

In [ ]:
clust_model = AgglomerativeClustering(n_clusters=4)
clust_model.fit(df)
labels = clust_model.labels_

print(metrics.calinski_harabasz_score(df, labels))
print(metrics.davies_bouldin_score(df, labels))

In [ ]:
13756.801318502405
1.3900641688585644

AHC separate goal-keeper 3 clusters

AHC separate goal-keeper 5 clusters

In [ ]:
#df => df_o, df_gk
clust_model = AgglomerativeClustering(n_clusters=5)
clust_model.fit(df_o)
labels = clust_model.labels_

df_concat = pd.concat([df_gk, df_o]);
gk_labels = np.full(shape = NUM_GOALKEEPER, fill_value=5, dtype=np.int)
labels_concat = np.concatenate((gk_labels, labels))
# print(type(gk_labels))
# print(gk_labels.shape)
# print(type(labels))
# print(labels.shape)

print(metrics.calinski_harabasz_score(df_concat, labels_concat))
print(metrics.davies_bouldin_score(df_concat, labels_concat))

In [ ]:
figure = plt.figure(figsize=(16, 9))
elbow_ch = figure.add_subplot(2,2,1) #elbow charts
elbow_db = figure.add_subplot(2,2,2)
elbow_sil = figure.add_subplot(2,2,3)
cost_ch=[]
cost_db=[]
cost_sil=[]

elbow_ch.set_ylabel('Calinski_Harabasz Score', fontsize = 15)
elbow_ch.set_xlabel('Number of Clusters', fontsize = 15)
elbow_ch.set_title('BIRCH Clustering: Elbow Chart', fontsize = 15)
    
elbow_db.set_ylabel('Davies-Bouldin Score', fontsize = 15)
elbow_db.set_xlabel('Number of Clusters', fontsize = 15)
elbow_db.set_title('BIRCH Clustering: Elbow Chart', fontsize = 15)

elbow_sil.set_ylabel('Silhouette Score', fontsize = 15)
elbow_sil.set_xlabel('Number of Clusters', fontsize = 15)
elbow_sil.set_title('BIRCH Clustering: Elbow Chart', fontsize = 15)
for num in range(2, 25):
    clust_model = Birch(n_clusters=num)
    clust_model.fit(df)
    labels = clust_model.labels_
    cost_ch.append(metrics.calinski_harabasz_score(df, labels))
    cost_db.append(metrics.davies_bouldin_score(df, labels))
    cost_sil.append(metrics.silhouette_score(df, labels, metric='euclidean'))
    elbow_ch.plot(cost_ch, 'bx-')
    elbow_db.plot(cost_db, 'bx-')
    elbow_sil.plot(cost_sil, 'bx-')

In [ ]:
figure = plt.figure(figsize=(16, 9))
elbow_ch = figure.add_subplot(2,2,1) #elbow charts
elbow_db = figure.add_subplot(2,2,2)
elbow_sil = figure.add_subplot(2,2,3)
cost_ch=[]
cost_db=[]
cost_sil=[]

elbow_ch.set_ylabel('Calinski_Harabasz Score', fontsize = 15)
elbow_ch.set_xlabel('Number of Clusters', fontsize = 15)
elbow_ch.set_title('Agglomerative Clustering: Elbow Chart', fontsize = 15)
    
elbow_db.set_ylabel('Davies-Bouldin Score', fontsize = 15)
elbow_db.set_xlabel('Number of Clusters', fontsize = 15)
elbow_db.set_title('Agglomerative Clustering: Elbow Chart', fontsize = 15)

elbow_sil.set_ylabel('Silhouette Score', fontsize = 15)
elbow_sil.set_xlabel('Number of Clusters', fontsize = 15)
elbow_sil.set_title('Agglomerative Clustering: Elbow Chart', fontsize = 15)
# clustering = AgglomerativeClustering().fit(X)
# for num in range(2, 25):
for num in range (2, 10):
    clust_model = AgglomerativeClustering(n_clusters=num)
    clust_model.fit(df)
    labels = clust_model.labels_
    cost_ch.append(metrics.calinski_harabasz_score(df, labels))
    cost_db.append(metrics.davies_bouldin_score(df, labels))
    cost_sil.append(metrics.silhouette_score(df, labels, metric='euclidean'))
    elbow_ch.plot(cost_ch, 'bx-')
    elbow_db.plot(cost_db, 'bx-')
    elbow_sil.plot(cost_sil, 'bx-')